## Notebook run brainlit code alongside matlab code from CloudReg package to test that they match

https://github.com/neurodata/CloudReg/blob/master/cloudreg/registration/transform_points.m

After some experiments, I observed that the main difference between the results comes from the interpolation. The affine transform, and euler method step are close to identical between cloudreg and me method (~10^-13 error). Also, the only difference in the code is the extrapolation method for the interpolator. CloudReg uses nearest extrapolation which is not an option in scipy. Indeed, when I use linear extrapolation in Cloudreg, results seem to match

`python -m cloudreg.scripts.transform_points --target_viz_link "https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=mDkeGmNCWVNCFg" --atlas_viz_link "https://ara.viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=HvyNDGaPsd1wyg" --affine_path "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/map_neurons/data/mapping-files/downloop_1_A.mat"  --velocity_path "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/map_neurons/data/mapping-files/downloop_1_v.mat"  --transformation_direction "atlas"`

Might need to run the matlab command manually

CloudReg:

Input: Ant->Post(263), Dors->Vent(159), R->L(227)
registration og_coords: 100 (D->V)x152(A->P)x134(R->L) registration size -> 100microns w/1000 microns pad on each side
diffeomorphism objects - y,x,z for inputs, 


In [2]:
from brainlit.map_neurons.map_neurons import (
    DiffeomorphismAction,
    transform_geometricgraph,
    compute_derivs,
    CloudReg_Transform,
)
from brainlit.algorithms.trace_analysis.fit_spline import (
    GeometricGraph,
    compute_parameterization,
    CubicHermiteChain,
)
from brainlit.utils.Neuron_trace import NeuronTrace

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy.interpolate import splev, splprep, CubicHermiteSpline
from scipy.spatial.distance import cosine
from scipy.stats import wilcoxon, pearsonr, linregress, norm
from tqdm import tqdm
from cloudvolume import CloudVolume
import seaborn as sns
from cloudvolume.exceptions import SkeletonDecodeError
import pickle
from copy import deepcopy
import networkx as nx
import os

In [3]:
root_dir = Path(os.path.abspath(""))
data_dir = os.path.join(root_dir, "data", "mapping-files")

In [5]:
velocity_path = os.path.join(data_dir, "downloop_1_v.mat")
affine_path = os.path.join(data_dir, "downloop_1_A.mat")

ct = CloudReg_Transform(velocity_path, affine_path)

integrating velocity field: 100%|██████████| 10/10 [00:23<00:00,  2.34s/it]


In [6]:
axons_path = "precomputed://file://" + os.path.join(data_dir, "axons")
vol = CloudVolume(axons_path)
shp = np.array(vol.shape)
res = np.array(vol.resolution) / 1000
origin = (shp[:3] - 1) * res / 2

skel = vol.skeleton.get(4)
coords = skel.vertices / 1000 - origin
coords = ct.apply_affine(coords)

## original vertices
The following are the vertex coordinates stored in the skeleton file

In [7]:
coords_skel = skel.vertices
print(coords_skel)

[[6680812.5 4213735.5 5570578. ]
 [6679242.  4213755.  5572043. ]
 [6660847.5 4216505.  5564594. ]
 ...
 [7051972.5 3908219.8 4851556.5]
 [7074937.5 4341430.5 5281373. ]
 [7068199.  4344958.  5270404.5]]


## neuroglancer
these are the voxel coordinates (of the highest res image) that you would use in neuroglancer

In [8]:
coords_ng = np.divide(coords_skel, 1000 * np.array([19.2, 19.2, 64]))
print(coords_ng)

[[347.95898438 219.46539062  87.04028125]
 [347.8771875  219.46640625  87.06317187]
 [346.91914063 219.60963542  86.94678125]
 ...
 [367.29023438 203.55311198  75.80557031]
 [368.48632812 226.11617187  82.52145312]
 [368.13536458 226.29989583  82.35007031]]


## centered
now, we changed units to microns and centered the image

In [9]:
coords_centered = coords_skel / 1000 - origin
print(coords_centered)

[[1468.0125      229.73535156  482.578125  ]
 [1466.4421875   229.75488281  484.04296875]
 [1448.04765625  232.50488281  476.59423828]
 ...
 [1839.17265625  -75.78027344 -236.44335938]
 [1862.1375      357.43066406  193.37304688]
 [1855.39921875  360.95800781  182.40429688]]


## affine
applied affine transform to coordinates

In [10]:
coords_affine = ct.apply_affine(coords_centered)
print(coords_affine)

[[ 524.30916956  737.44099749 1371.16397248]
 [ 525.84386745  737.47461986 1369.48174897]
 [ 518.37428854  739.81165879 1349.2803704 ]
 ...
 [-210.42621759  335.60447696 1773.2164223 ]
 [ 214.64419038  898.37724199 1790.00024407]
 [ 203.32006118  901.97601536 1782.37493956]]


## complete transform

In [11]:
coords_transform = ct.evaluate(coords_affine)
print(coords_transform)

[[ 528.77887889  737.17408455 1368.58931551]
 [ 530.31465855  737.20482804 1366.90567841]
 [ 522.86942046  739.54004881 1346.75897028]
 ...
 [-205.95088358  336.61983272 1772.45870123]
 [ 218.73289536  898.69899859 1787.58085665]
 [ 207.42380189  902.30522157 1779.99932288]]


## ng atlas voxel space

In [12]:
coords_ng_atlas = np.divide(coords_transform, 10) + np.array([1320, 800, 1140]) / 2
coords_ng_atlas

array([[712.87788789, 473.71740846, 706.85893155],
       [713.03146586, 473.7204828 , 706.69056784],
       [712.28694205, 473.95400488, 704.67589703],
       ...,
       [639.40491164, 433.66198327, 747.24587012],
       [681.87328954, 489.86989986, 748.75808567],
       [680.74238019, 490.23052216, 747.99993229]])

## cloudreg transform point process summary:
- voxel coordinates
- convert to microns
- center (target) image

## result: 
[712.87788789, 473.71740846, 706.85893155] vs cloudreg [712.8782348632812, 473.7189025878906, 706.8578491210938]